<a href="https://colab.research.google.com/github/Juniormoraess/Pyomo---OPL/blob/master/Problema%20da%20produ%C3%A7%C3%A3o%20de%20multiperiodos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## Instalação do pyomo e do solver para o colab ##
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [0]:
## Modelo matemático ##
## Máx 50x1 + 45x2 + 55x3 + 48x4 + 52x5 + 50x6 + 8(e1 + e2 + e3 + e4 + e5 + e6) ##
##                         ##
## S.a  e0 + x1 - e1 = 100 ##
##      e1 + x2 - e2 = 250 ##
##      e2 + x2 - e3 = 190 ##
##      e3 + x3 - e4 = 140 ##
##      e4 + x4 - e5 = 220 ##
##      e5 + x5 - e6 = 110 ##
##      x1 + x2 + x3 + x4 + x5 + x6 = 1010 ##


## Importando a biblioteca pyomo ##
import pyomo.environ as pyEnv

## Dados do problema ##
custo_producao = [50, 45, 55, 48, 52, 50]
custo_estoque = 8.0
demanda = [100, 250, 190, 140, 220, 110]
estoque_inicial = 0.0
m = len(custo_producao)
n = len(demanda)

## Declarando o modelo através da biblioteca ##
modelo = pyEnv.ConcreteModel()

## Criando Indices ##
modelo.Indices = pyEnv.RangeSet(1, m)
modelo.Indices2 = pyEnv.RangeSet(0, n)

## Criando as variáveis ##
modelo.var_producao = pyEnv.Var(modelo.Indices, within = pyEnv.NonNegativeReals)
modelo.var_estoque = pyEnv.Var(modelo.Indices2, within = pyEnv.NonNegativeReals)

## Criando a Função Objetivo ##
modelo.Objetivo = pyEnv.Objective(expr = sum(custo_producao[i-1]*modelo.var_producao[i] + custo_estoque*modelo.var_estoque[i-1] for i in modelo.Indices), sense = pyEnv.minimize)
modelo.var_estoque[0] = estoque_inicial

## Criando as restrições ##
def rest1(modelo, i): 
  return modelo.var_estoque[i-1] + modelo.var_producao[i] - modelo.var_estoque[i] == demanda[i-1]

modelo.rest = pyEnv.Constraint(modelo.Indices, rule = rest1)
modelo.rest2 = pyEnv.Constraint(expr = sum(modelo.var_producao[i] for i in modelo.Indices) == sum(demanda[i-1] for i in modelo.Indices))

## Chamando o Solver ##
solver = pyEnv.SolverFactory('glpk', executable = '/usr/bin/glpsol')
result_objetivo = solver.solve(modelo, tee = True)

## Printando o resultado ##
lista = list(modelo.var_producao.keys())
lista2 = list(modelo.var_estoque.keys())
print()
print()
print('Variaveis de produção: ')
print()
for i in lista:
  print(i, '---', modelo.var_producao[i]())
print()
print('Variaveis de estoque: ')
print()
for i in lista2:
  print(i, '---', modelo.var_estoque[i]())
print()
print('Valor da função objetivo =', modelo.Objetivo())
